In [1]:
# Reading values that my program found and collecting basic statistics

import pandas as pd

df = pd.read_csv(".\\results_first_run_interval_100.csv")

# calculate the total execution time
total_execution_time_sec = round(df['Execution time'].sum(), 2)
minutes = total_execution_time_sec // 60
hours = minutes // 60

print("All STC test were calculated in: {} seconds, {} minutes, hours {}".format(total_execution_time_sec, minutes, hours))

# extract the test name and rename columns
df['Test name'] = df['Test name'].apply(lambda x: x.split('/')[-1])
df = df.rename(columns={'Execution time': 'Execution time Leonid'})
df = df.rename(columns={'Result': 'Result Leonid'})

df

All STC test were calculated in: 709.79 seconds, 11.0 minutes, hours 0.0


,Test name,Execution time Leonid,Result Leonid,Values
0,STC_0001.smt2,0.003764,SAT,x = -100 y = 1 z = 100
1,STC_0002.smt2,0.323468,SAT,x = -47 y = -24 z = 49
2,STC_0003.smt2,0.573846,SAT,x = -5 y = 4 z = 4
3,STC_0004.smt2,1.253370,Not SAT,NaN
4,STC_0005.smt2,1.565967,Not SAT,NaN
...,...,...,...,...
995,STC_0996.smt2,0.549865,SAT,x = -7 y = 2 z = 11
996,STC_0997.smt2,0.556982,SAT,x = -3 y = 8 z = 8
997,STC_0998.smt2,0.533815,SAT,x = -9 y = -1 z = 12
998,STC_0999.smt2,0.530791,SAT,x = -9 y = 0 z = 12


In [2]:
# Reading values from original cvc5 executable and collecting basic statistics
import pandas as pd

df_orig = pd.read_csv(".\\results_original_cvc5_timeout_1_min.csv")

# calculate the total execution time
total_execution_time_sec = round(df_orig['Execution time'].sum(), 2)
minutes = total_execution_time_sec // 60
hours = minutes // 60

print("All STC test were calculated in: {} seconds, {} minutes, hours {}".format(total_execution_time_sec, minutes, hours))
print("Inmortant note, that timeout was set to 1 minute!")

# extract the test name and rename columns
df_orig['Test name'] = df_orig['Test name'].apply(lambda x: x.split('/')[-1])
df_orig = df_orig.rename(columns={'Execution time': 'Execution time CVC5'})
df_orig = df_orig.rename(columns={'Result': 'Result CVC5'})


df_orig

All STC test were calculated in: 55544.7 seconds, 925.0 minutes, hours 15.0
Inmortant note, that timeout was set to 1 minute!


,Test name,Execution time CVC5,Result CVC5,Values
0,STC_0001.smt2,22.307753,sat,NaN
1,STC_0002.smt2,0.249582,sat,NaN
2,STC_0003.smt2,0.378245,sat,NaN
3,STC_0004.smt2,60.026129,NaN,NaN
4,STC_0005.smt2,60.028352,NaN,NaN
...,...,...,...,...
995,STC_0996.smt2,60.024707,NaN,NaN
996,STC_0997.smt2,60.023573,NaN,NaN
997,STC_0998.smt2,60.024111,NaN,NaN
998,STC_0999.smt2,60.030542,NaN,NaN


In [3]:
# Mmerging 2 datframes to compare and make analysis

resulting_df = pd.merge(df, df_orig, on='Test name')

# Droping values calculaed from CVC5, because they are not calculated
resulting_df = resulting_df.drop('Values_y', axis=1)

# Values calculated by my solver could be used later for verification
resulting_df = resulting_df.rename(columns={'Values_x': 'Values Leonid'})

# strip any leading or trailing whitespace from the "Result CVC5" column
resulting_df['Result Leonid'] = resulting_df['Result Leonid'].str.strip().str.lower()

# count the number of rows where the "Result Leonid" column and the "Result CVC5" column have the same value
same_results_counter = sum(resulting_df['Result Leonid'] == resulting_df['Result CVC5'])
print("In total there are {} cases when the programs gave the same results".format(same_results_counter))


# Define the lambda function to calculate missing values
calculate_missing = lambda col: col.isna().sum()
sat_counter = lambda col: sum(1 for val in col if val == 'sat')

# Apply the lambda function to column 'B'
missing_values_cvc = resulting_df['Result CVC5'].pipe(calculate_missing)



sat_count_Leonid = sat_counter(resulting_df['Result Leonid'])
sat_count_cvc = sat_counter(resulting_df['Result CVC5'])

print("Number of 'sat' values in column: Result Leonid: {}".format( sat_count_Leonid))
print("Number of 'sat' values in column: Result CVC5: {}".format(sat_count_cvc))

print("There are {} missing values in column Result CVC5.".format(missing_values_cvc))

resulting_df


In total there are 144 cases when the programs gave the same results
Number of 'sat' values in column: Result Leonid: 615
Number of 'sat' values in column: Result CVC5: 144
There are 856 missing values in column Result CVC5.


,Test name,Execution time Leonid,Result Leonid,Values Leonid,Execution time CVC5,Result CVC5
0,STC_0001.smt2,0.003764,sat,x = -100 y = 1 z = 100,22.307753,sat
1,STC_0002.smt2,0.323468,sat,x = -47 y = -24 z = 49,0.249582,sat
2,STC_0003.smt2,0.573846,sat,x = -5 y = 4 z = 4,0.378245,sat
3,STC_0004.smt2,1.253370,not sat,NaN,60.026129,NaN
4,STC_0005.smt2,1.565967,not sat,NaN,60.028352,NaN
...,...,...,...,...,...,...
995,STC_0996.smt2,0.549865,sat,x = -7 y = 2 z = 11,60.024707,NaN
996,STC_0997.smt2,0.556982,sat,x = -3 y = 8 z = 8,60.023573,NaN
997,STC_0998.smt2,0.533815,sat,x = -9 y = -1 z = 12,60.024111,NaN
998,STC_0999.smt2,0.530791,sat,x = -9 y = 0 z = 12,60.030542,NaN


## Report on Comparison of STC Test Results: My SMT Solver vs. Original CVC5 Solver

### Introduction:
I developed an SMT solver that solves constraints by testing values in the range from -100 to 100. I used this solver to run 1000 Math Problems STC tests and compared my results to those of the original CVC5 solver. In this report, I will discuss the results of this comparison.

### Comparison of Results:
Out of the 1000 tests, my program gave the same result as the original CVC5 solver in 144 cases. This suggests that my solver is performing reasonably well, given that it is based on a brute-force approach and tests values only in a limited interval. Also, important to note that if my solver didn't find the solution inside the interval it will return NOT SAT, meanwhile if cvc is not able to solve a constraint inside a 1-minute timeout it will return NaN

### Time Comparison:
The time it took for my solver to run all 1000 tests was 11 minutes, while the original CVC5 solver took 15 hours. However, I set a timeout of 1 minute for the original CVC5 solver, after which it terminated if a value was not found. This indicates that my solver is faster than the original CVC5 solver in finding values for the range tested.

### Conclusion:
In conclusion, the results of the comparison show that my SMT solver is performing well, given its brute-force approach and limited testing range. Additionally, my solver is faster than the original CVC5 solver in finding values within the range tested. However, it is important to note that further testing is required to evaluate the performance of my solver on more complex problems and larger testing ranges. 


In [5]:
# Reading values that my program found and collecting basic statistics

import pandas as pd

df = pd.read_csv(".\\results_second_run_interval_300.csv")

# calculate the total execution time
total_execution_time_sec = round(df[' Execution time'].sum(), 2)
minutes = total_execution_time_sec // 60
hours = minutes // 60

print("All STC test were calculated in: {} seconds, {} minutes, hours {}".format(total_execution_time_sec, minutes, hours))

# extract the test name and rename columns
df['Test name'] = df['Test name'].apply(lambda x: x.split('/')[-1])
df = df.rename(columns={' Execution time': 'Execution time Leonid'})
df = df.rename(columns={'Result': 'Result Leonid'})

df

All STC test were calculated in: 19581.2 seconds, 326.0 minutes, hours 5.0


,Test name,Execution time Leonid,Result,Values
0,STC_0001.smt2,0.030689,SAT,x = -300 y = 1 z = 300
1,STC_0002.smt2,7.177578,SAT,x = -161 y = -54 z = 163
2,STC_0003.smt2,15.041955,SAT,x = -5 y = 4 z = 4
3,STC_0004.smt2,31.527554,Not SAT,NaN
4,STC_0005.smt2,31.542761,Not SAT,NaN
...,...,...,...,...
995,STC_0996.smt2,18.222798,SAT,x = -7 y = 2 z = 11
996,STC_0997.smt2,8.747455,SAT,x = -153 y = 65 z = 149
997,STC_0998.smt2,16.570406,SAT,x = -9 y = -1 z = 12
998,STC_0999.smt2,7.658770,SAT,x = -168 y = 111 z = 150


In [10]:
# Reading values from original cvc5 executable and collecting basic statistics
import pandas as pd

df_orig = pd.read_csv(".\\results_original_cvc5_time_out_3_min.csv")

# calculate the total execution time
total_execution_time_sec = round(df_orig['Execution time'].sum(), 2)
minutes = total_execution_time_sec // 60
hours = minutes // 60

print("All STC test were calculated in: {} seconds, {} minutes, hours {}".format(total_execution_time_sec, minutes, hours))
# print("Inmortant note, that timeout was set to 1 minute!")

# extract the test name and rename columns
df_orig['Test name'] = df_orig['Test name'].apply(lambda x: x.split('/')[-1])
df_orig = df_orig.rename(columns={'Execution time': 'Execution time CVC5'})
df_orig = df_orig.rename(columns={'Result': 'Result CVC5'})


df_orig

All STC test were calculated in: 155392.03 seconds, 2589.0 minutes, hours 43.0


,Test name,Execution time CVC5,Result CVC5
0,STC_0001.smt2,21.429063,sat
1,STC_0002.smt2,0.260718,sat
2,STC_0003.smt2,0.232070,sat
3,STC_0004.smt2,180.027488,cvc5 interrupted by SIGTERM.
4,STC_0251.smt2,180.030957,cvc5 interrupted by SIGTERM.
...,...,...,...
995,STC_0996.smt2,180.025757,cvc5 interrupted by SIGTERM.
996,STC_0997.smt2,180.026950,cvc5 interrupted by SIGTERM.
997,STC_0998.smt2,180.026174,cvc5 interrupted by SIGTERM.
998,STC_0999.smt2,99.801605,sat


In [25]:
# Mmerging 2 datframes to compare and make analysis

resulting_df = pd.merge(df, df_orig, on='Test name')

# # Droping values calculaed from CVC5, because they are not calculated
# resulting_df = resulting_df.drop('Values_y', axis=1)

# # Values calculated by my solver could be used later for verification
resulting_df = resulting_df.rename(columns={' Values': 'Values Leonid'})
resulting_df = resulting_df.rename(columns={' Result': 'Result Leonid'})

# count the number of rows where the "Result Leonid" column and the "Result CVC5" column have the same value
resulting_df['Result Leonid'] = resulting_df['Result Leonid'].str.strip().str.lower()

same_results_counter = sum(resulting_df['Result Leonid'] == resulting_df['Result CVC5'])
print("In total there are {} cases when the programs gave the same results".format(same_results_counter))


# Define the lambda function to calculate missing values
calculate_missing = lambda col: sum(1 for val in col if val == 'cvc5 interrupted by SIGTERM.')
sat_counter = lambda col: sum(1 for val in col if val == 'sat')

# # Apply the lambda function to column 'B'
missing_values_cvc = resulting_df['Result CVC5'].pipe(calculate_missing)



sat_count_Leonid = sat_counter(resulting_df['Result Leonid'])
sat_count_cvc = sat_counter(resulting_df['Result CVC5'])

print("Number of 'sat' values in column: Result Leonid: {}".format( sat_count_Leonid))
print("Number of 'sat' values in column: Result CVC5: {}".format(sat_count_cvc))

print("There are {} cvc5 interrupted by SIGTERM values in column Result CVC5.".format(missing_values_cvc))

resulting_df


In total there are 169 cases when the programs gave the same results
Number of 'sat' values in column: Result Leonid: 637
Number of 'sat' values in column: Result CVC5: 169
There are 831 cvc5 interrupted by SIGTERM values in column Result CVC5.


,Test name,Execution time Leonid,Result Leonid,Values Leonid,Execution time CVC5,Result CVC5
0,STC_0001.smt2,0.030689,sat,x = -300 y = 1 z = 300,21.429063,sat
1,STC_0002.smt2,7.177578,sat,x = -161 y = -54 z = 163,0.260718,sat
2,STC_0003.smt2,15.041955,sat,x = -5 y = 4 z = 4,0.232070,sat
3,STC_0004.smt2,31.527554,not sat,NaN,180.027488,cvc5 interrupted by SIGTERM.
4,STC_0005.smt2,31.542761,not sat,NaN,180.032515,cvc5 interrupted by SIGTERM.
...,...,...,...,...,...,...
995,STC_0996.smt2,18.222798,sat,x = -7 y = 2 z = 11,180.025757,cvc5 interrupted by SIGTERM.
996,STC_0997.smt2,8.747455,sat,x = -153 y = 65 z = 149,180.026950,cvc5 interrupted by SIGTERM.
997,STC_0998.smt2,16.570406,sat,x = -9 y = -1 z = 12,180.026174,cvc5 interrupted by SIGTERM.
998,STC_0999.smt2,7.658770,sat,x = -168 y = 111 z = 150,99.801605,sat


In [73]:
# Starting analysis of STC problems, timeout 3 min


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

original_cvc = pd.read_csv(".\\results_original_cvc5_time_out_3_min.csv")

original_cvc = original_cvc.rename(columns={'Result': 'Result CVC'})
original_cvc = original_cvc.rename(columns={'Execution time': 'Execution time CVC'})



my_solver =  pd.read_csv(".\\results_second_run_interval_300.csv")


# extract the test name and rename columns
my_solver['Test name'] = my_solver['Test name'].apply(lambda x: x.split('/')[-1])
my_solver = my_solver.rename(columns={' Execution time': 'Execution time my solver'})
my_solver = my_solver.rename(columns={' Result': 'Result my solver'})

my_solver['Result my solver'] = my_solver['Result my solver'].str.strip().str.lower()

# merge the two dataframes based on the 'Test name' column
merged_df = pd.merge(my_solver, original_cvc, on='Test name')


# define a function to process the result for each test
def process_result(row):
    if row['Result CVC'] == 'sat':
        return 'Match' if row['Result my solver'] == 'sat' else 'Mismatch'
    elif row['Result CVC'] == 'unsat':
        return 'Match' if row['Result my solver'] == 'unsat' else 'Mismatch'
    else:
        return 'Mismatch'

# process the results for each test and add a column to the merged dataframe
merged_df['Result comparison'] = merged_df.apply(process_result, axis=1)

same_results_counter = sum(merged_df['Result my solver'] == merged_df['Result CVC'])

# # create a bar chart of the result comparison for each test
# sns.countplot(x='Result comparison', data=merged_df)
# plt.title('Result Comparison')
# plt.show()


# Result consistency
same_results_counter = sum(merged_df['Result my solver'] == merged_df['Result CVC'])
total_tests = merged_df.shape[0]
result_consistency = same_results_counter / total_tests
print("The two solvers gave the same result for {:.2f}% of the tests.".format(result_consistency * 100))


# Performance comparison
avg_cvc_time = merged_df['Execution time CVC'].mean()
median_cvc_time = merged_df['Execution time CVC'].median()
avg_my_solver_time = merged_df['Execution time my solver'].mean()
median_my_solver_time = merged_df['Execution time my solver'].median()
print("On average, the original solver took {:.2f} seconds and my solver took {:.2f} seconds.".format(avg_cvc_time, avg_my_solver_time))
print("The median execution time for the original solver was {:.2f} seconds and for my solver it was {:.2f} seconds.".format(median_cvc_time, median_my_solver_time))


df_head = merged_df.head(5)

df_tail = merged_df.tail(5)

# concatenate the dataframes vertically
small_df = pd.concat([df_head, df_tail])

# convert the concatenated dataframe to latex
latex = small_df.to_latex(index=False)


print(latex)

counter = 0
for index, row in merged_df.iterrows():
    if row['Result CVC'] == 'cvc5 interrupted by SIGTERM.' and not pd.isna(row[' Values']):
        counter += 1

print("")
print(f"There are {counter} cases when original cvc5 had a timeout, but my solver was able to find the result.")

sat_counter = lambda col: sum(1 for val in col if val == 'sat')
sat_count_cvc = sat_counter(merged_df['Result CVC'])

print(f"There are {sat_count_cvc} cases when cvc solved the problem")


merged_df


The two solvers gave the same result for 16.90% of the tests.
On average, the original solver took 155.39 seconds and my solver took 19.58 seconds.
The median execution time for the original solver was 180.04 seconds and for my solver it was 15.43 seconds.
\begin{tabular}{lrllrll}
\toprule
    Test name &  Execution time my solver & Result my solver &                     Values &  Execution time CVC &                   Result CVC & Result comparison \\
\midrule
STC\_0001.smt2 &                  0.030689 &              sat &    x = -300 y = 1 z = 300  &           21.429063 &                          sat &             Match \\
STC\_0002.smt2 &                  7.177578 &              sat &  x = -161 y = -54 z = 163  &            0.260718 &                          sat &             Match \\
STC\_0003.smt2 &                 15.041955 &              sat &        x = -5 y = 4 z = 4  &            0.232070 &                          sat &             Match \\
STC\_0004.smt2 &                 

C:\Users\Leonid\AppData\Local\Temp\ipykernel_20216\1784037278.py:74: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex = small_df.to_latex(index=False)


,Test name,Execution time my solver,Result my solver,Values,Execution time CVC,Result CVC,Result comparison
0,STC_0001.smt2,0.030689,sat,x = -300 y = 1 z = 300,21.429063,sat,Match
1,STC_0002.smt2,7.177578,sat,x = -161 y = -54 z = 163,0.260718,sat,Match
2,STC_0003.smt2,15.041955,sat,x = -5 y = 4 z = 4,0.232070,sat,Match
3,STC_0004.smt2,31.527554,not sat,NaN,180.027488,cvc5 interrupted by SIGTERM.,Mismatch
4,STC_0005.smt2,31.542761,not sat,NaN,180.032515,cvc5 interrupted by SIGTERM.,Mismatch
...,...,...,...,...,...,...,...
995,STC_0996.smt2,18.222798,sat,x = -7 y = 2 z = 11,180.025757,cvc5 interrupted by SIGTERM.,Mismatch
996,STC_0997.smt2,8.747455,sat,x = -153 y = 65 z = 149,180.026950,cvc5 interrupted by SIGTERM.,Mismatch
997,STC_0998.smt2,16.570406,sat,x = -9 y = -1 z = 12,180.026174,cvc5 interrupted by SIGTERM.,Mismatch
998,STC_0999.smt2,7.658770,sat,x = -168 y = 111 z = 150,99.801605,sat,Match
